## HW 3 - Introduction LP Formulations

In this first homework assignment we will work on 5 different LP forumulations in Gurobi. 

For each of them, a skeleton python code is provided, and you are tasked with filling in some additional bits and piecees. 

As a little help, the output of each cell is what it should be if you fill in the right code in each cell (where it is currently missing).

In [2]:
# Install Gurobi Python Interface

import gurobipy as gp

### Problem 1

Problem 1 in this assignment is a "classical first linear programming problem": a diet optimization problem. 

We are given different types of food, that each come with a (financial) cost and with nutritional information (amount of calories, protein, fat, and sodium). In addition, we are told in what limits our nutritional intake should be for each of the nutrients (e.g., at least 91g of protein).

Below, I will walk you through the optimization problem that aims to minimize the financial cost of the food items we consume whilst guaranteeing that we are within the limits of calories,protein, fat, and sodium.

In [3]:
# Data for the problem

# Data 1 : The below dictionary shows the minimum and maximum ranges of each nutrient
categories, minNutrition, maxNutrition = gp.multidict({
    'calories': [1800, 2200],
    'protein':  [91, gp.GRB.INFINITY],
    'fat':      [0, 65],
    'sodium':   [0, 1779]})

# Data 2 : The below dictionary shows the cost of each available food
foods, cost = gp.multidict({
    'hamburger': 2.49,
    'chicken':   2.89,
    'hot dog':   1.50,
    'fries':     1.89,
    'macaroni':  2.09,
    'pizza':     1.99,
    'salad':     2.49,
    'milk':      0.89,
    'ice cream': 1.59})

# Data 3 : The below dictionary shows the the amount of distinct nutrition available in each food
nutritionValues = {
    ('hamburger', 'calories'): 410,
    ('hamburger', 'protein'):  24,
    ('hamburger', 'fat'):      26,
    ('hamburger', 'sodium'):   730,
    ('chicken',   'calories'): 420,
    ('chicken',   'protein'):  32,
    ('chicken',   'fat'):      10,
    ('chicken',   'sodium'):   1190,
    ('hot dog',   'calories'): 560,
    ('hot dog',   'protein'):  20,
    ('hot dog',   'fat'):      32,
    ('hot dog',   'sodium'):   1800,
    ('fries',     'calories'): 380,
    ('fries',     'protein'):  4,
    ('fries',     'fat'):      19,
    ('fries',     'sodium'):   270,
    ('macaroni',  'calories'): 320,
    ('macaroni',  'protein'):  12,
    ('macaroni',  'fat'):      10,
    ('macaroni',  'sodium'):   930,
    ('pizza',     'calories'): 320,
    ('pizza',     'protein'):  15,
    ('pizza',     'fat'):      12,
    ('pizza',     'sodium'):   820,
    ('salad',     'calories'): 320,
    ('salad',     'protein'):  31,
    ('salad',     'fat'):      12,
    ('salad',     'sodium'):   1230,
    ('milk',      'calories'): 100,
    ('milk',      'protein'):  8,
    ('milk',      'fat'):      2.5,
    ('milk',      'sodium'):   125,
    ('ice cream', 'calories'): 330,
    ('ice cream', 'protein'):  8,
    ('ice cream', 'fat'):      10,
    ('ice cream', 'sodium'):   180}

Before going ahead, try to build a mathematical formulation for the problem above. 

In this assignment , we do not expect you to turn in your mathematical formulation, but none the less, it is always advised to build you mathematical model first then build the model.

We use this data below when we define our optimization problem.

In [4]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet1")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: minimizing the cost of the food
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

# Fourth, we set constraints that enforce for each nutrition category
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories:
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), 
               # this sum captures the amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint 
        # a name is not required but it is a good habit to 
        # give each variable/constraint a unique name
# We could have written the constraints also by writing
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) >= minNutrition[c])
# and
# m.addConstr(sum(nutritionValues[f, c] * buy[f] for f in foods) <= maxNutrition[c])
# We will see situations below in which this notation is easier to write.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23


That's it. Probably that was not that hard, right? 

Now we can tell Gurobi to find us an optimal solution. When Gurobi solves, it gives us a lot of information that we may not care about (at least, not right now). 

So before solving, we define a function that ends up printing the interesting bits of the optimal solution for us.

In [5]:
def printSolution(m):
    if m.status == gp.GRB.OPTIMAL: # to check whether an optimal solution was found
        buyx = m.getAttr('x', buy)
        print('Cost: %g' %(sum(cost[f] * buyx[f] for f in foods)))
        print('\nNutrients:')
        for c in categories:
            print('%s %g'%(c, sum(buyx[f]*nutritionValues[f,c] for f in foods)))
        print('\nBuy:')
        for f in foods:
            if buy[f].x > 0.0001:
                print('%s %g' % (f, buyx[f]))
        
    else:
        print('No solution')

In [6]:
# Now we solve, and get a bunch of info we don't necessarily understand
# If we want to stop gurobi from printing that info wee can add 
# m.setParam( 'OutputFlag', False ) to the code to stop Gurobi from printing
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x33ddb849
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.182886111e+01


In [7]:
# Now we print the interesting aspects of our solution:
printSolution(m)

Cost: 11.8289

Nutrients:
calories 1800
protein 91
fat 59.0559
sodium 1779

Buy:
hamburger 0.604514
milk 6.97014
ice cream 2.59132


I hope everything above has made sense.  

#### A) 
Below, we will try to set up a slightly different optimization problem. Now, rather than aiming to minimize cost, we try to build muscle, i.e., we need to maximize our protein intake. 

The cell below has most of the protein maximization model written out (same as above), with "only the objective missing". Add the new objective to the optimization problem and have gurobi solve it.

In [9]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet2")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: maximizing the amount of protein in our diet.
# This is almost the same as before, except for                                                                 [Delete this for Question File]
# and rather than MINIMIZE we write MAXIMIZE (or we add a minus before the sum)                                 [Delete this for Question File]


### Enter the objective function below ###
m.setObjective(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MAXIMIZE)                       #[Delete this for Question File]


# could also be: m.setObjective(- sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MINIMIZE)    [Delete this for Question File]

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
m.optimize()
printSolution(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xfdf84afa
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e+00, 3e+01]
  Bounds range     [7e+01, 2e+03]
  RHS range        [7e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1312500e+02   5.734416e+02   0.000000e+00      0s
       3    1.0672516e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.067251613e+02
Cost: 13.2912

Nutrients:
calories 1800
protein 106.725
fat 48.5452
sodium 1779

Buy:
milk 11.3148
ice cream 2.02581


#### B) 
Similar to part (A), we still want to maximize our protein intake. However, we now have an added budget constraint: our cost can be at most 12.5. Write out and solve this new optimization problem (you should be able to copy-paste the cell from question A, and add just one line with the new budget constraint).

In [10]:
# First, we define a variable that contains the optimization model
m = gp.Model("diet3")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective as in part A: maximizing the amount of protein in our diet.
m.setObjective(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods), gp.GRB.MAXIMIZE)                   # [Delete this for Question File]


# Fourth, we set constraints that enforce for each nutrition category
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # this is the lower bound (from the data)
               maxNutrition[c], # this is the upper bound (from the data)
               c) # this is the name for the constraint

#Alternate formulation of the constraints
'''                                                                                                         Delete this for Question File
for c in categories: #
    LHS = 0
    for f in foods:
        LHS += nutritionValues[f, c] * buy[f]
    m.addRange(LHS, # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
'''

# Finally, you need to add a constraint that bounds the cost of items bought by 12.5
# To do so, we define the cost as it was in the original objective: [Delete this for Question File]         [Delete this for Question File]
# sum(buy[f]*cost[f] for f in foods) and use the constraint syntax m.addConstr("INSERT COST" <= 12.5)       [Delete this for Question File]

### Enter the objective function below ###
m.addConstr(sum(buy[f]*cost[f] for f in foods) <= 12.5)                                                     #[Delete this for Question File]
    
m.optimize()
printSolution(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x6250ef36
Coefficient statistics:
  Matrix range     [9e-01, 2e+03]
  Objective range  [4e+00, 3e+01]
  Bounds range     [7e+01, 2e+03]
  RHS range        [1e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 5 rows, 10 columns, 46 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1312500e+02   6.103916e+02   0.000000e+00      0s
       4    9.8216855e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.821685513e+01
Cost: 12.5

Nutrients:
calories 1800
protein 98.2169
fat 54.2321
sodium 1779

Buy:
hamburger 0.32708
milk 8.96408
ice cream 2.33178

#### C)

Explain in words why the protein we get is lower in question B than it was in question A. Can you identify the budget we need to consume the maximum amount of protein? (either implement this, or just explain how you would do it)

Answer C):

Enter your answer here : 

[Delete for question file]
In question B we maximized our protein subject to not spending more than 12.5, and got to 98.22. In question A, we got to 106 without the cost constraint. This is because the cost constraint cuts all solutions with more than 98.22 protein off from the feasible region.

In order to find the budget that allows us to consume the maximum amount of protein, we can solve an optimization problem that has as a constraint that protein is as high as possible (i.e., at 106.7) but aims to minimize the budget subject to that. Implemented below, this shows us that the cost from A, 13.29 was optimal subject to the constraint that the protein intake is maximized) (notice: the cost now is slightly lower, because we constrained the protein to be >=106.7, whereas in A it is 106.725...).

In [ ]:
                                                                                                        #[Delete entire block for Question File]

# First, we define a variable that contains the optimization model
m = gp.Model("diet4")

# Second, we add decision variables to the model that capture for each 
# food item the quantity bought
buy = {}
for f in foods:
    buy[f] = m.addVar(name=f)

# Third, we set our objective: minimize our budget
m.setObjective(sum(buy[f]*cost[f] for f in foods), gp.GRB.MINIMIZE)

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    m.addRange(sum(nutritionValues[f, c] * buy[f] for f in foods), # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint
    
'''
Alternate formulation of constraints

# Fourth, we set constraints that enforce for each nutrition category 
# (calories, protein, fat, sodium) that we be in the feasible range
for c in categories: # 
    LHS = 0
    for f in foods:
        LHS += nutritionValues[f, c] * buy[f]
    m.addRange(LHS, # again: amount of each nutrient
               minNutrition[c], # again: lower bound (from the data)
               maxNutrition[c], # again: upper bound (from the data)
               c) # name for constraint


'''

# Fifth, we force protein to be at the maximum:
m.addConstr(sum(buy[f]*nutritionValues[f, 'protein'] for f in foods) >= 106.7)
m.optimize()
printSolution(m)

#### Problem 2

Consider the following problem Amazon may face (let's not kid ourselves: the variation Amazon faces is 100x more complex; but this is a simplified version): they have 4 different suppliers that each have some amount of inventory of a particular SKU. They also have 9 different orders for that SKU with individual fulfillment centers asking for one shipment of that SKU. Given the costs of shipping from each supplier to each fulfillment center, decide which supplier should ship to which fulfillment center, such that the resulting shipping costs are minimized.

(PS: yes, we're assuming that sending items fractionally will be allowed, at a fractional cost, for the purpose of this question! Though, it turns out, that that won't be an issue!)

In [12]:
# Define supplier inventories
supplier_inventories = {'w1': 4, 'w2': 3, 'w3': 2, 'w4':3}
# the dictionary below holds the cost of shipping from each supplier to each center.
# We assume that the shipment costs are linear in the amount shipped,
supplier_center_cost = {'w1': [3, 5, 6, 2, 4, 6, 3, 6, 4],
                           'w2': [6, 5, 8, 3, 5, 6, 3, 5, 2],
                           'w3': [3, 5, 3, 2, 4, 7, 3, 6, 4],
                           'w4': [1, 4, 1, 4, 1, 8, 5, 1, 3]}
# e.g., the cost of one shipment from supplier 'w1' to the first center is 3
# whereas the cost of half a shipment from supplier 'w3' to the final center is 1/2*4=2

#Define the optimization model
m = gp.Model("shipping")

# Define decision variables to indicate for each supplier/center how much is shipped
shipments = {}
for w in supplier_inventories:
    for center in range(9):
        shipments[w,center] = m.addVar()                                                             #Enter command to add decision variable here [Delete the m.addvar() for Question File]
        
# Next, we set our objective: minimizing the cost of all shipments
m.setObjective(sum(sum(shipments[w,center]*supplier_center_cost[w][center]                          #[Delete for Question File]
                for w in supplier_inventories) for center in range(9)),                             #[Delete for Question File]
                gp.GRB.MINIMIZE)                                                                    #[Delete for Question File]

'''                                                                                                 [Delete this for Question File]

Alternate formulation of objective
# Next, we set our objective: minimizing the cost of all shipments
objective = 0
for w in supplier_inventories:
    for center in range(9):
        objective += shipments[w,center]*supplier_center_cost[w][center]
m.setObjective(objective, gp.GRB.MINIMIZE)

'''

# Set a constraint that the amount of inventory shipped from a given supplier is at most its inventory
for w in supplier_inventories:
    # We want the sum over all decision variables that leave w to be at most supplier_inventories[w] #[Delete for Question File]
    m.addConstr(sum(shipments[w, center] for center in range(9))                                     #[Delete for Question File]
                 <= supplier_inventories[w])                                                         #[Delete for Question File]

# Set a constraint that each center receives one shipment of inventory
for center in range(9):
    # We want the sum over all decision variables that go to center to be 1                          #[Delete for Question File]
    m.addConstr(sum(shipments[w, center] for w in supplier_inventories)                              #[Delete for Question File]
                 == 1)                                                                               #[Delete for Question File]

'''                                                                                                  [Delete this for Question File]

Alternate formulation of constraints
# Set a constraint that the amount of inventory shipped from a given supplier is at most its inventory
for w in supplier_inventories:
    # We want the sum over all decision variables that leave w to be at most supplier_inventories[w]
    LHS = 0
    for center in range(9):
        LHS += shipments[w, center]
    m.addConstr( LHS <= supplier_inventories[w])
# Set a constraint that each center receives one shipment of inventory
for center in range(9):
    # We want the sum over all decision variables that go to center to be 1
    LHS = 0
    for w in supplier_inventories:
        LHS+= shipments[w, center]
    m.addConstr(LHS == 1)


'''
    
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 36 columns and 72 nonzeros
Model fingerprint: 0xd2c55f7f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 13 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+01   2.000000e+00   0.000000e+00      0s
       3    2.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.400000000e+01


Below we print for each center the supplier from which the shipment is made and the fraction shipped 

In [15]:
for center in range(9):
    for w in supplier_inventories:
        if shipments[w,center].x>0: print(w, center, supplier_center_cost[w][center],shipments[w,center].x)

w1 0 3 1.0
w2 1 5 1.0
w4 2 1 1.0
w1 3 2 1.0
w4 4 1 1.0
w1 5 6 1.0
w1 6 3 1.0
w4 7 1 1.0
w2 8 2 1.0


#### Problem 3

In this question we need to decide on a production schedule for a factory. The factory is tasked with producing 3 different goods, and it has 3 different machines; each machine can be tasked to work on any of the goods, but machines are faster at producing some goods than they are at others. Our goal is to minimize the time until all our orders are fulfilled.

In the cell below we provide you with the data for the problem:

In [16]:
# we have products 'p1', 'p2', 'p3', and we need to
# produce (respectively) 100, 200, 400 units of each
product_demand = {'p1': 100, 'p2': 200, 'p3': 400}
machines = ['m1', 'm2', 'm3']
# we have three machines, 'm1', 'm2', and 'm3' and
# the number of products of each type that can be produced
# by each of the machines is given in the dictionary below
machine_speed = {('m1','p1'):2, ('m1','p2'):1, ('m1','p3'):3,
                 ('m2','p1'):2, ('m2','p2'):2, ('m2','p3'):4.5,
                 ('m3','p1'):2, ('m3','p2'):3, ('m3','p3'):4}
# e.g., machine 'm1' can produce 2 units of type 'p1' but only
# 1 unit of type 'p2' per hour

Hint: In setting up your optimization problem it will be useful to define 10 different variables: 9 for each pair of machine and product ("how many hours does machine i spend on product j?"), and 1 for the objective ("what is the maximum time any of the machines is working?"), which we refer to as makespan.

In [17]:
m = gp.Model("production_schedule")

time_spent = {}
# First add a decision variable capturing the time each machine spends on each product
for machine,p in machine_speed:
    time_spent[(machine,p)] =  m.addVar(lb= 0, name = machine+p)                                        #Delete the RHS for Question File]

# Next, add constraints to ensure we spend enough time (by respective machines) on 
# each product so that enough units of each products are completed
for p in product_demand:
     m.addConstr(sum(time_spent[machine,p]*machine_speed[machine,p] for machine in ['m1','m2','m3'])    #Delete this for Question File]
                   >= product_demand[p])                                                                #Delete this RHS for Question File]
     
'''                                                                                                     [Delete this for Question File]
Alternate formulation of constraints
# Next, add constraints to ensure we spend enough time (by respective machines) on 
# each product so that enough units of each products are completed
for p in product_demand:
    LHS = 0
    for machine in ['m1','m2','m3']:
        LHS+= time_spent[machine,p]*machine_speed[machine,p]
    m.addConstr( LHS >= product_demand[p])

'''
    
makespan = m.addVar(lb=0)   # makespan will be the objective, the maximum time any
                            # machine works in total. Notice that lb is the lower bound
                            # on a variable, so we do not allow the makespan to be negative

# Add a constraint that the makespan variable is no less than the time each 
# machine spends individually
for machine in ['m1', 'm2', 'm3']:
     m.addConstr(sum(time_spent[machine,p] for p in product_demand)<= makespan)                         #Delete the RHS for Question File]

'''                                                                                                     [Delete this for Question File]
Alternate formulation of constraints
# Add a constraint that the makespan variable is no less than the time each 
# machine spends individually
for machine in ['m1', 'm2', 'm3']:
    LHS=0
    for p in product_demand:
        LHS += time_spent[machine,p]
    m.addConstr( LHS <= makespan)

'''

# Finally, set the objective to be the minimization of the makespan. 
m.setObjective(makespan, gp.GRB.MINIMIZE)                                                               #Delete the RHS for Question File]

# Since we have only constrained the makespan to be bounded below by the time spent by any 
# of 'm1', 'm2', 'm3' (and nothing else), and since the objective should minimize the 
# makespan, the optimal  solution will have the makespan be equal to the maximum of 
# time spent by any of the machines. Specifically, there is no reason (constraint) 
# forcing the makespan variable to be any larger than the maximum total time spent, 
# and the minimization objective is "pushing" it down until one of the inequalities 
# holds with equality.


m.optimize()

print("The optimal schedule is:")
for x in time_spent:
    if time_spent[x].x>0:
        print("""Machine %s works on product %s for %g hours, producing %g units"""%(x[0], 
                                            x[1], time_spent[x].x, 
                                            machine_speed[x]*time_spent[x].x))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x3dfcf0e0
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 9 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+02   0.000000e+00      0s
       5    7.1014493e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.101449275e+01
The optimal schedule is:
Machine m1 works on product p1 for 50 hours, producing 100 units
Machine m1 works on product p3 for 21.0145

#### Problem 4

The industry operates a large online platform with millions of transactions every day. In order to process all the payments, the company partners with 2 different financial providers. In a nutshell, the providers have different cost structures for processing each payment. The goal of this optimization problem is to decide which payments to send to which provider.

- Provider A charges a proportional fee that is 2\% of the cost of the transaction, and requires the average payment routed to them to have a transaction value of at least 8\$
- Provider B charges a fixed fee of \$.20 per payment

Below is a skeleton of a linear program that decides for each transaction which provider to send it to. Fill in the objective and the constraint (for provider A).

In [18]:
# The list below has all transactions for one day.
Transactions = [5, 10, 14, 20, 6, 12, 16, 10, 8, 9, 12, 15]
num_transactions = len(Transactions)

In [19]:
# First, we define a variable that contains the optimization model
m = gp.Model("transactions")

# Second, we add decision variables that capture for each 
# transaction which provider we send it to. A good way of doing this is
# by creating a variable that is either 0 or 1, where 0 indicates it is being 
# sent to A and 1 indicates it is being sent to B.
provider = {}
for k in range(num_transactions):
    provider[k] = m.addVar(lb=0,ub=1)
    
# Third, you need to add a constraint that ensures that the average value 
# of transactions sent to provider A is at least $8. This is tricky!

# A transaction is sent to A if (1-provider[k])=1. So the average value sent to A is                                                #Delete this for Question File]
# sum((1-provider[k])*Transactions[k] for k in range(num_transactions)) / sum((1-provider[k]) for k in range(num_transactions))     #Delete this for Question File]
# and we want this to be >=8. The above is nonlinear but by multiplying by                                                          #Delete this for Question File]
# sum((1-provider[k]) for k in range(num_transactions) on both sides we get                                                         #Delete this for Question File]

m.addConstr(sum((1-provider[k])*Transactions[k] for k in range(num_transactions))                                                   #Delete this for Question File]
           >= 8*sum((1-provider[k]) for k in range(num_transactions)))                                                              #Delete this for Question File]

'''                                                                                                                                 [Delete this for Question File]
Alternate formulation of constraints
LHS, RHS = 0, 0
for k in range(num_transactions):
    LHS += (1-provider[k])*Transactions[k]
    RHS += 8*(1-provider[k])

m.addConstr(LHS >= RHS)

'''

# Last, define the objective. This is also not so easy:
# You want a term for each provider that contributes 
# .02*Transactions[k] if provider[k]=0 and .2 if provider[k]=1

# In other words, multpliy .2 by provider[k] (it is 0 if provider[k]=0, and .2 if provider[k]=1)                                    #Delete this for Question File]
# and multiply .02*Transactions[k] by (1-provider[k]) (it is 0 if provider[k]=1, and                                                #Delete this for Question File]
# .02*Transactions[k] if provider[k]=0)                                                                                             #Delete this for Question File]
m.setObjective(sum([ (1-provider[k]) * .02 * Transactions[k] + provider[k]*.2                                                       #[Delete this for Question File]
            for k in range(num_transactions)]), gp.GRB.MINIMIZE)                                                                    #[Delete this for Question File]

'''                                                                                                                                 [Delete this for Question File]   
Alternate formulation of objective
objective = 0
for k in range(num_transactions):
    objective += (1-provider[k]) * .02 * Transactions[k] + provider[k]*.2 

m.setObjective( objective, gp.GRB.MINIMIZE)
'''


m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 12 columns and 11 nonzeros
Model fingerprint: 0x8e4738c5
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+01]
Presolve removed 1 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1600000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.160000000e+00


#### Problem 5

In this problem we try to find the optimal schedule of workers to different shifts in a store. Suppose our store is open 24/7, and the number of workers needed to staff the store is given below.

In [20]:
staff_required = {0: 8, 1: 8, 2: 9, 3: 8, 4: 8, 5: 9, 
                  6: 10, 7: 12, 8: 12, 9: 12, 10: 10, 
                  11: 10, 12: 12, 13: 12, 14: 10, 15: 10,
                  16: 9, 17: 10, 18: 12, 19: 12, 20: 12,
                  21: 10, 22: 9, 23: 8}

A worker scheduled to show up at a given time, stays for 4h. Find the number of workers to show up at each hour of the day that minimizes the total number of workers that need are needed over the course of the day while fulfilling the staff needs above.

Hint: the python function \% may be useful in coding up the constraints!

In [23]:
# First, we define a variable that contains the optimization model
m = gp.Model("store_shifts")

# Second, let's create one variable for each hour in which a shift can start
starts = {}
for k in range(24):
    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k))                                                 #[Delete this for Question File]

# Third, define the objective: the total number of workers needed
m.setObjective(sum([starts[k] for k in range(24)]), gp.GRB.MINIMIZE)                                            #[Delete this for Question File]
    
# Fourth, add a constraint for each hour that ensures that sufficiently
# many workers are scheduled to work at that hour.
# Notice that the number of workers working at hour k are the ones scheduled to start                           #[Delete this for Question File]
# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure          #[Delete this for Question File]
# that, say, -2 counts as 22, i.e., 10PM) #[Delete this for Question File]                                      #[Delete this for Question File]

for k in range(24): 
    m.addConstr( (starts[(k-3)%24] + starts[(k-2)%24] + starts[(k-1)%24] + starts[k] ) >= staff_required[k])    #[Delete this for Question File]

m.optimize()

''''                                                                                                             [Delete this for Question File]
Alternate formulation 

# First, we define a variable that contains the optimization model
m = gp.Model("store_shifts")

# Second, let's create one variable for each hour in which a shift can start
starts = {}
for k in range(24):
    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k)) 

# Third, define the objective: the total number of workers needed
objective = 0
for k in range(24):
    objective+=starts[k]
m.setObjective(objective, gp.GRB.MINIMIZE)
    
# Fourth, add a constraint for each hour that ensures that sufficiently
# many workers are scheduled to work at that hour.
# Notice that the number of workers working at hour k are the ones scheduled to start
# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure 
# that, say, -2 counts as 22, i.e., 10PM)
for k in range(24):
    m.addConstr(starts[(k-3)%24]+starts[(k-2)%24]+starts[(k-1)%24]+starts[k]>=staff_required[k])

m.optimize()
'''




Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 24 columns and 96 nonzeros
Model fingerprint: 0x7e92d4fd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+01]
Presolve time: 0.01s
Presolved: 24 rows, 24 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.420000e+02   0.000000e+00      0s
      22    6.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.100000000e+01


'\'\nAlternate formulation \n\n# First, we define a variable that contains the optimization model\nm = gp.Model("store_shifts")\n\n# Second, let\'s create one variable for each hour in which a shift can start\nstarts = {}\nfor k in range(24):\n    starts[k] = m.addVar(lb=0, ub=gp.GRB.INFINITY, name=str(k)) \n\n# Third, define the objective: the total number of workers needed\nobjective = 0\nfor k in range(24):\n    objective+=starts[k]\nm.setObjective(objective, gp.GRB.MINIMIZE)\n    \n# Fourth, add a constraint for each hour that ensures that sufficiently\n# many workers are scheduled to work at that hour.\n# Notice that the number of workers working at hour k are the ones scheduled to start\n# at k-3, k-2, k-1 and k (where k-3, k-2, k-1 should be treated with mod 24 (in python: %24) to ensure \n# that, say, -2 counts as 22, i.e., 10PM)\nfor k in range(24):\n    m.addConstr(starts[(k-3)%24]+starts[(k-2)%24]+starts[(k-1)%24]+starts[k]>=staff_required[k])\n\nm.optimize()\n'

In [24]:
for k in range(len(starts)): print(k, starts[k].x, staff_required[k])

0 7.0 8
1 0.0 8
2 2.0 9
3 0.0 8
4 6.0 8
5 1.0 9
6 4.0 10
7 1.0 12
8 6.0 12
9 1.0 12
10 2.0 10
11 1.0 10
12 8.0 12
13 1.0 12
14 0.0 10
15 1.0 10
16 7.0 9
17 2.0 10
18 2.0 12
19 1.0 12
20 7.0 12
21 0.0 10
22 1.0 9
23 0.0 8


End of Home work 3